In [1]:
import pandas as pd
import numpy as np 
import os.path as op

from necessity_functions import NecessityUnivariate, NecessityBivariate
from robust_set_sampling import RobustCredalSetUnivariate, RobustCredalSetBivariate
from copulas import min_copula

### Declare marginal possibilities and orders

In [2]:
a = {'x1':0.2, "x2":1, "x3":0.2}
order_a = pd.DataFrame(data=[2,1], index=["x2", "x1,x2,x3"], columns=["order"])
order_a_p = pd.DataFrame(data=[1,2,3], index=["x1","x2","x3"], columns=["order"])

b = {'y1':0.7, "y2":1}
order_b = pd.DataFrame(data=[1,2], index=["y2", "y1,y2"], columns=["order"])
order_b_p = pd.DataFrame(data=[1,2], index=["y1","y2"], columns=["order"])

### Necessity functions

In [3]:
nec_x = NecessityUnivariate(a, order_a)
nec_y = NecessityUnivariate(b, order_b)

nec_xy = NecessityBivariate(nec_x, nec_y, min_copula)

In [4]:
rob_x = RobustCredalSetUnivariate(nec_x)
rob_y = RobustCredalSetUnivariate(nec_y)

rob_xy = RobustCredalSetBivariate(rob_x, rob_y, order_a_p, order_b_p, min_copula)

# Other tests

In [3]:
output_dir = "/work/scratch/malinoro/simulation_copula/out"
multi = pd.MultiIndex.from_tuples(list(
        zip(["poss", "poss", "poss", "poss", "poss", "poss", "poss", "poss", "focal_sets", "focal_sets"],
            ["x1", "x2", "x3", "x4", "y1", "y2", "y3", "y4", "X", "Y"])), names=["Object", "Space"])
final_df = pd.DataFrame(columns=multi)

In [4]:
final_df.to_csv(op.join(output_dir, "test.csv"))

In [5]:
read_df = pd.read_csv(op.join(output_dir, "test.csv"), header=[0,1])
read_df

Empty DataFrame
Columns: [(Object, Space), (poss, x1), (poss, x2), (poss, x3), (poss, x4), (poss, y1), (poss, y2), (poss, y3), (poss, y4), (focal_sets, X), (focal_sets, Y)]
Index: []

In [34]:
final_df.loc[0, :] = [1, 0.2, 0.1, 0.7, 0.5, 0.2, 1, 0.6, "x1<x1,x4<x1,x2,x4<x1,x2,x3,x4", "y3<y3,y4<y1,y3,y4<y1,y2,y3,y4"]
final_df.to_csv(op.join(output_dir, "test.csv"), mode='a', header=False)


In [35]:
read_df = pd.read_csv(op.join(output_dir, "test.csv"), header=[0,1])
read_df

Object poss                                                      focal_sets  \
   Space   x1   x2   x3   x4   y1   y2 y3   y4                              X   
0      0    1  0.2  0.1  0.7  0.5  0.2  1  0.6  x1<x1,x4<x1,x2,x4<x1,x2,x3,x4   

                                  
                               Y  
0  y3<y3,y4<y1,y3,y4<y1,y2,y3,y4

In [36]:
final_df.drop(axis=0, labels=[0], inplace=True)
final_df.loc[1, :] = [1, 0.5, 0.1, 0.7, 0.5, 0.2, 1, 0.9, "x1<x1,x4<x1,x2,x4<x1,x2,x3,x4", "y3<y3,y4<y1,y3,y4<y1,y2,y3,y4"]
final_df.to_csv(op.join(output_dir, "test.csv"), mode='a', header=False)

In [37]:
read_df = pd.read_csv(op.join(output_dir, "test.csv"), header=[0,1])
read_df

Object poss                                                      focal_sets  \
   Space   x1   x2   x3   x4   y1   y2 y3   y4                              X   
0      0    1  0.2  0.1  0.7  0.5  0.2  1  0.6  x1<x1,x4<x1,x2,x4<x1,x2,x3,x4   
1      1    1  0.5  0.1  0.7  0.5  0.2  1  0.9  x1<x1,x4<x1,x2,x4<x1,x2,x3,x4   

                                  
                               Y  
0  y3<y3,y4<y1,y3,y4<y1,y2,y3,y4  
1  y3<y3,y4<y1,y3,y4<y1,y2,y3,y4

In [46]:
final_df.loc[2, [("poss", k) for k in a.keys()]] = a.values()

In [47]:
final_df

Object poss                                                        focal_sets  \
Space    x1   x2   x3   x4   y1   y2   y3   y4                              X   
1         1  0.5  0.1  0.7  0.5  0.2    1  0.9  x1<x1,x4<x1,x2,x4<x1,x2,x3,x4   
2       0.2    1  0.2  NaN  NaN  NaN  NaN  NaN                            NaN   

Object                                 
Space                               Y  
1       y3<y3,y4<y1,y3,y4<y1,y2,y3,y4  
2                                 NaN

In [56]:
order_x = pd.DataFrame(columns=["order"], index=["a","b","c"], data=[2,3,1])
"<".join(order_x.sort_values(axis=0, by=["order"]).index)

'c<a<b'

In [57]:
order_x

,order
a,2
b,3
c,1
